Tenemos los CSV, vamos a lanzar consultas en SQL, por lo que crearemos una Base de Datos, pasaremos los CSVs a 
tablas dentro de la Base de Datos y así podremos lanzar consultas

In [47]:
# Importar las bibliotecas necesarias
import pandas as pd  # Para manejar archivos CSV y bases de datos
import sqlite3  # Para interactuar con SQLite
import os  # Para comprobar la existencia de archivos

## Comprobar tipo de datos y formato

In [48]:
import pandas as pd

# Lista de archivos CSV en tu proyecto
csv_files = [
    "customers csv.csv",
    "dim_geography.csv",
    "dim_product.csv",
    "dim_territory.csv",
    "fact_sales.csv"
]

# Analizar cada archivo
for file in csv_files:
    print(f"\n🔹 Análisis del archivo: {file}\n")

    # Detectar si es un CSV con delimitador `;`
    with open(file, 'r') as f:
        first_line = f.readline()
        delimiter = ';' if ';' in first_line else ','

    # Cargar CSV con el delimitador correcto
    df = pd.read_csv(file, delimiter=delimiter)

    # Crear DataFrame con nombre de columna, ejemplo de dato y tipo de dato
    analysis = pd.DataFrame({
        "Columna": df.columns,
        "Ejemplo": df.iloc[0].astype(str),  # Convertimos la primera fila a string
        "Tipo de Dato": df.dtypes.values
    })

    # Mostrar tabla en formato más limpio
    from tabulate import tabulate
    print(tabulate(analysis, headers="keys", tablefmt="pretty"))
    print("-" * 50)




🔹 Análisis del archivo: customers csv.csv

+-------------------+-------------------+------------+--------------+
|                   |      Columna      |  Ejemplo   | Tipo de Dato |
+-------------------+-------------------+------------+--------------+
|    customer_id    |    customer_id    |   11000    |    int64     |
|   geography_id    |   geography_id    |     26     |    int64     |
|     birthdate     |     birthdate     | 08/04/1986 |    object    |
|   maritalstatus   |   maritalstatus   |     M      |    object    |
|      gender       |      gender       |     M      |    object    |
| datefirstpurchase | datefirstpurchase | 22/07/2016 |    object    |
+-------------------+-------------------+------------+--------------+
--------------------------------------------------

🔹 Análisis del archivo: dim_geography.csv

+-----------------------------+-----------------------------+-----------------+--------------+
|                             |           Columna           |     

# NUEVAS COLUMNAS A EUROS Y NUEVOS CSV VERSIONADOS

In [49]:
import pandas as pd

# Lista de archivos CSV con los que trabajaremos
csv_files = [
    "dim_product.csv",
    "fact_sales.csv"
]

# Tasa de conversión de Rupiah a Euro
rupiah_to_euro = 0.000055

# Analizar cada archivo y agregar las columnas convertidas
for file in csv_files:
    print(f"\n🔹 Procesando archivo: {file}\n")

    # Detectar si es un CSV con delimitador `;` o `,`
    with open(file, 'r') as f:
        first_line = f.readline()
        delimiter = ';' if ';' in first_line else ','

    # Cargar CSV con el delimitador correcto
    df = pd.read_csv(file, delimiter=delimiter)

    # Comprobamos que los valores sean numéricos antes de aplicar la conversión
    if file == "dim_product.csv":
        # Convertir las columnas cost y normal_price a euros si existen
        if 'cost' in df.columns:
            df['cost_euro'] = (df['cost'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'normal_price' in df.columns:
            df['normal_price_euro'] = (df['normal_price'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)

    elif file == "fact_sales.csv":
        # Convertir las columnas unitprice_rupiah, totalprice_rupiah, totalcost_rupiah, shippingprice_rupiah a euros
        if 'unitprice_rupiah' in df.columns:
            df['unitprice_euro'] = (df['unitprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'totalprice_rupiah' in df.columns:
            df['totalprice_euro'] = (df['totalprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'totalcost_rupiah' in df.columns:
            df['totalcost_euro'] = (df['totalcost_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)
        if 'shippingprice_rupiah' in df.columns:
            df['shippingprice_euro'] = (df['shippingprice_rupiah'].apply(lambda x: x * rupiah_to_euro if isinstance(x, (int, float)) else x)).round(2)

    # Guardar el archivo con las columnas convertidas a euros
    df.to_csv(f"converted_{file}", index=False)

    # Mostrar las primeras filas del archivo procesado
    print(f"Primeras filas de {file}:")
    print(df.head(10))




🔹 Procesando archivo: dim_product.csv

Primeras filas de dim_product.csv:
   product_id               product_name          model_name  color  \
0         210  HL Road Frame - Black, 58       HL Road Frame  Black   
1         211    HL Road Frame - Red, 58       HL Road Frame    Red   
2         480        Patch Kit/8 Patches           Patch kit    NaN   
3         529             Road Tire Tube      Road Tire Tube    NaN   
4         477      Water Bottle - 30 oz.        Water Bottle    NaN   
5         528         Mountain Tire Tube  Mountain Tire Tube    NaN   
6         530          Touring Tire Tube   Touring Tire Tube    NaN   
7         484      Bike Wash - Dissolver           Bike Wash    NaN   
8         224               AWC Logo Cap         Cycling Cap  Multi   
9         223               AWC Logo Cap         Cycling Cap  Multi   

  size_range     cost  normal_price       sub_category     category  \
0   54-58 CM  11000.0       11000.0        Road Frames   Components   
1

# 1. Conectar o crear la base de datos SQLite

In [50]:

# Si el archivo 'sales_data.db' no existe, SQLite lo creará automáticamente
conn = sqlite3.connect("sales_data.db")
print("Conexión establecida con la base de datos.")

Conexión establecida con la base de datos.


# 2. Definir los archivos CSV y los nombres de las tablas correspondientes y cargarlos a tablas

In [51]:

import os
import pandas as pd
import sqlite3

# Establecer la conexión con la base de datos SQLite
conn = sqlite3.connect('sales_data.db')

# Crear un cursor
cursor = conn.cursor()

# Creamos un diccionario donde la clave es el nombre de la tabla en la base de datos
# y el valor es el nombre del archivo CSV
csv_files = {
    "customers": "customers csv.csv",
    "dim_customer": "dim_customer.csv",
    "dim_geography": "dim_geography.csv",
    "dim_product": "converted_dim_product.csv",
    "territory": "dim_territory.csv",   # Corregido
    "fact_sales": "converted_fact_sales.csv"   # Corregido
}

# Cargar cada archivo CSV en su tabla correspondiente en SQLite
for table_name, file_name in csv_files.items():
    if os.path.exists(file_name):  # Verificamos que el archivo exista
        df = pd.read_csv(file_name)  # Leer el archivo CSV en un DataFrame
        df.to_sql(table_name, conn, if_exists="replace", index=False)  # Guardar en SQLite
        print(f"Tabla '{table_name}' cargada correctamente con {len(df)} registros.")
    else:
        print(f"El archivo '{file_name}' no se encontró. Verifica su nombre y ubicación.")

# Cerrar la conexión
conn.close()


DatabaseError: Execution failed on sql 'DROP TABLE "customers"': database is locked

# Exportamos en la terminal sqlite3 sales_data.db .dump > export.sql

Para exportar la base de datos completa a un archivo SQL (incluyendo la estructura de las tablas y los datos) desde la terminal de SQLite, sigue estos pasos:

In [ ]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect("sales_data.db")
cursor = conn.cursor()

# Consultar las tablas creadas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Obtener y mostrar los resultados
tables = cursor.fetchall()
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])

# Cerrar la conexión
conn.close()


Tablas en la base de datos:
dim_territory
dim_customer
dim_territory_new
customers
dim_geography
dim_product
territory
fact_sales


In [ ]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect("sales_data.db")
cursor = conn.cursor()

# Consultar las tablas creadas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Obtener y mostrar los resultados
tables = cursor.fetchall()
print("Tablas en la base de datos:")
for table in tables:
    table_name = table[0]
    print(f"\nTabla: {table_name}")

    # Obtener la información de la tabla
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    # Mostrar la información de cada columna (nombre y tipo de dato)
    for column in columns:
        column_name = column[1]
        column_type = column[2]
        print(f"Columna: {column_name}, Tipo de Dato: {column_type}")

# Cerrar la conexión
conn.close()


Tablas en la base de datos:

Tabla: dim_territory
Columna: territory_id, Tipo de Dato: INTEGER
Columna: region, Tipo de Dato: TEXT
Columna: country, Tipo de Dato: TEXT
Columna: groups, Tipo de Dato: TEXT

Tabla: dim_customer
Columna: customer_id, Tipo de Dato: INTEGER
Columna: geography_id, Tipo de Dato: INTEGER
Columna: customer_name, Tipo de Dato: TEXT
Columna: birthdate, Tipo de Dato: TEXT
Columna: maritalstatus, Tipo de Dato: TEXT
Columna: gender, Tipo de Dato: TEXT
Columna: datefirstpurchase, Tipo de Dato: TEXT

Tabla: dim_territory_new
Columna: territory_id, Tipo de Dato: INT
Columna: region, Tipo de Dato: VARCHAR(255)
Columna: country, Tipo de Dato: VARCHAR(255)
Columna: groups, Tipo de Dato: VARCHAR(255)

Tabla: customers
Columna: customer_id;geography_id;birthdate;maritalstatus;gender;datefirstpurchase, Tipo de Dato: TEXT

Tabla: dim_geography
Columna: geography_id, Tipo de Dato: INTEGER
Columna: city, Tipo de Dato: TEXT
Columna: state_province_code, Tipo de Dato: TEXT
Columna

# AAAAAH

In [ ]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect("sales_data.db")
cursor = conn.cursor()

# Cambiar el tipo de datos de las tablas (en SQLite)
# Se usa un ALTER TABLE o se puede crear una nueva tabla con el tipo de datos corregido y luego transferir los datos

# 1. Cambiar la tabla `dim_territory`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_territory_new (
        territory_id INT,
        region VARCHAR(255),
        country VARCHAR(255),
        groups VARCHAR(255)
    );
''')

cursor.execute('''
    INSERT INTO dim_territory_new (territory_id, region, country, groups)
    SELECT territory_id, region, country, groups FROM dim_territory;
''')

cursor.execute('DROP TABLE IF EXISTS dim_territory;')
cursor.execute('ALTER TABLE dim_territory_new RENAME TO dim_territory;')

# 2. Cambiar la tabla `customers`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers_new (
        customer_id INT,
        geography_id INT,
        birthdate DATE,
        maritalstatus VARCHAR(10),
        gender VARCHAR(10),
        datefirstpurchase DATE
    );
''')

cursor.execute('''
    INSERT INTO customers_new (customer_id, geography_id, birthdate, maritalstatus, gender, datefirstpurchase)
    SELECT customer_id, geography_id, birthdate, maritalstatus, gender, datefirstpurchase FROM customers;
''')

cursor.execute('DROP TABLE IF EXISTS customers;')
cursor.execute('ALTER TABLE customers_new RENAME TO customers;')

# 3. Cambiar la tabla `dim_customer`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_customer_new (
        customer_id INT,
        geography_id INT,
        customer_name VARCHAR(255),
        birthdate DATE,
        maritalstatus VARCHAR(10),
        gender VARCHAR(10),
        datefirstpurchase DATE
    );
''')

cursor.execute('''
    INSERT INTO dim_customer_new (customer_id, geography_id, customer_name, birthdate, maritalstatus, gender, datefirstpurchase)
    SELECT customer_id, geography_id, customer_name, birthdate, maritalstatus, gender, datefirstpurchase FROM dim_customer;
''')

cursor.execute('DROP TABLE IF EXISTS dim_customer;')
cursor.execute('ALTER TABLE dim_customer_new RENAME TO dim_customer;')

# 4. Cambiar la tabla `dim_geography`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_geography_new (
        geography_id INT,
        city VARCHAR(255),
        state_province_code VARCHAR(255),
        state_province_name VARCHAR(255),
        country_region_code VARCHAR(255),
        english_country_region_name VARCHAR(255),
        postal_code VARCHAR(255)
    );
''')

cursor.execute('''
    INSERT INTO dim_geography_new (geography_id, city, state_province_code, state_province_name, country_region_code, english_country_region_name, postal_code)
    SELECT geography_id, city, state_province_code, state_province_name, country_region_code, english_country_region_name, postal_code FROM dim_geography;
''')

cursor.execute('DROP TABLE IF EXISTS dim_geography;')
cursor.execute('ALTER TABLE dim_geography_new RENAME TO dim_geography;')

# 5. Cambiar la tabla `dim_product`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_product_new (
        product_id INT,
        product_name VARCHAR(255),
        model_name VARCHAR(255),
        color VARCHAR(255),
        size_range VARCHAR(255),
        cost DECIMAL(10,2),
        normal_price DECIMAL(10,2),
        sub_category VARCHAR(255),
        category VARCHAR(255),
        cost_euro DECIMAL(10,2),
        normal_price_euro DECIMAL(10,2)
    );
''')

cursor.execute('''
    INSERT INTO dim_product_new (product_id, product_name, model_name, color, size_range, cost, normal_price, sub_category, category, cost_euro, normal_price_euro)
    SELECT product_id, product_name, model_name, color, size_range, cost, normal_price, sub_category, category, cost_euro, normal_price_euro FROM dim_product;
''')

cursor.execute('DROP TABLE IF EXISTS dim_product;')
cursor.execute('ALTER TABLE dim_product_new RENAME TO dim_product;')

# 6. Cambiar la tabla `territory`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS territory_new (
        territory_id INT,
        region VARCHAR(255),
        country VARCHAR(255),
        groups VARCHAR(255)
    );
''')

cursor.execute('''
    INSERT INTO territory_new (territory_id, region, country, groups)
    SELECT territory_id, region, country, groups FROM territory;
''')

cursor.execute('DROP TABLE IF EXISTS territory;')
cursor.execute('ALTER TABLE territory_new RENAME TO territory;')

# 7. Cambiar la tabla `fact_sales`
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_sales_new (
        order_detail_id VARCHAR(255),
        order_date DATE,
        product_id INT,
        customer_id INT,
        territory_id INT,
        sales_order_number VARCHAR(255),
        sales_order_line_number INT,
        quantity INT,
        unitprice_rupiah DECIMAL(10,2),
        totalprice_rupiah DECIMAL(10,2),
        totalcost_rupiah DECIMAL(10,2),
        shippingprice_rupiah DECIMAL(10,2),
        unitprice_euro DECIMAL(10,2),
        totalprice_euro DECIMAL(10,2),
        totalcost_euro DECIMAL(10,2),
        shippingprice_euro DECIMAL(10,2)
    );
''')

cursor.execute('''
    INSERT INTO fact_sales_new (order_detail_id, order_date, product_id, customer_id, territory_id, sales_order_number, sales_order_line_number, quantity, unitprice_rupiah, totalprice_rupiah, totalcost_rupiah, shippingprice_rupiah, unitprice_euro, totalprice_euro, totalcost_euro, shippingprice_euro)
    SELECT order_detail_id, order_date, product_id, customer_id, territory_id, sales_order_number, sales_order_line_number, quantity, unitprice_rupiah, totalprice_rupiah, totalcost_rupiah, shippingprice_rupiah, unitprice_euro, totalprice_euro, totalcost_euro, shippingprice_euro FROM fact_sales;
''')

cursor.execute('DROP TABLE IF EXISTS fact_sales;')
cursor.execute('ALTER TABLE fact_sales_new RENAME TO fact_sales;')

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()

print("Los tipos de datos han sido modificados correctamente para MySQL.")


OperationalError: no such column: customer_id

# Cambiar algunos términos de SQLite para que sea compatible con MySQL
> • Reemplaza todas las ocurrencias de AUTOINCREMENT por AUTO_INCREMENT (MySQL usa AUTO_INCREMENT). <br>
> • Si ves PRAGMA o BEGIN TRANSACTION, elimínalos (MySQL no los usa). <br>
> • Asegúrate de que las FOREIGN KEY tengan ON DELETE CASCADE si quieres que se comporten igual en MySQL. <br>